<a href="https://colab.research.google.com/github/geantrindade/ConvNet-Performance-Prediction/blob/master/model_meta_extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Imports**

In [0]:
import torch
import torch.nn as nn

import torchvision
import torchvision.models as models

import statistics
import numpy as np
from scipy import stats
import pandas as pd

# **Utils**

### Pre-trained models

In [0]:
def call_constructor(convNet_name):
  if(convNet_name == "alexnet"): 
    return models.alexnet() 
    
  elif(convNet_name == "inception_v3"): 
    return models.inception_v3()
  
  elif(convNet_name == "googlenet"): 
    return models.googlenet()

  elif(convNet_name == "mobilenet_v2"): 
    return models.mobilenet_v2()

  elif(convNet_name == "vgg11"): 
    return models.vgg11()

  elif(convNet_name == "vgg11_bn"): 
    return models.vgg11_bn()

  elif(convNet_name == "vgg13"): 
    return models.vgg13()

  elif(convNet_name == "vgg13_bn"): 
    return models.vgg13_bn()

  elif(convNet_name == "vgg16"): 
    return models.vgg16()

  elif(convNet_name == "vgg16_bn"): 
    return models.vgg16_bn()

  elif(convNet_name == "vgg19"): 
    return models.vgg19()

  elif(convNet_name == "vgg19_bn"): 
    return models.vgg19_bn()

  elif(convNet_name == "resnet18"): 
    return models.resnet18()

  elif(convNet_name == "resnet34"): 
    return models.resnet34()

  elif(convNet_name == "resnet50"): 
    return models.resnet50()

  elif(convNet_name == "resnet101"): 
    return models.resnet101()

  elif(convNet_name == "resnet152"): 
    return models.resnet152()

  elif(convNet_name == "squeezenet1_0"): 
    return models.squeezenet1_0()

  elif(convNet_name == "squeezenet1_1"): 
    return models.squeezenet1_1()

  elif(convNet_name == "densenet121"): 
    return models.densenet121()

  elif(convNet_name == "densenet161"): 
    return models.densenet161()

  elif(convNet_name == "densenet169"): 
    return models.densenet169()

  elif(convNet_name == "densenet201"): 
    return models.densenet201()

  elif(convNet_name == "shufflenet_v2_x0_5"): 
    return models.shufflenet_v2_x0_5()

  elif(convNet_name == "shufflenet_v2_x1_0"): 
    return models.shufflenet_v2_x1_0()

  elif(convNet_name == "resnext50_32x4d"): 
    return models.resnext50_32x4d()

  elif(convNet_name == "resnext101_32x8d"): 
    return models.resnext101_32x8d()

  elif(convNet_name == "wide_resnet50_2"): 
    return models.wide_resnet50_2()

  elif(convNet_name == "wide_resnet101_2"): 
    return models.wide_resnet101_2()

  elif(convNet_name == "mnasnet0_5"): 
    return models.mnasnet0_5()

  elif(convNet_name == "mnasnet1_0"): 
    return models.mnasnet1_0()

### Get Layers

In [0]:
def get_conv2d_layers(convNet) -> list:
  return [layer for layer in convNet if type(layer) == nn.Conv2d]

def get_maxpool2d_layers(convNet) -> list:
  return [layer for layer in convNet if type(layer) == nn.MaxPool2d]

def get_adaptive_avgpool2d_layers(convNet) -> list:
  return [layer for layer in convNet if type(layer) == nn.AdaptiveAvgPool2d]

def get_batchnorm2d_layers(convNet) -> list:
  return [layer for layer in convNet if type(layer) == nn.BatchNorm2d]

def get_dropout_layers(convNet) -> list:
  return [layer for layer in convNet if type(layer) == nn.Dropout]

def get_linear_layers(convNet) -> list:
  return [layer for layer in convNet if type(layer) == nn.Linear]

def get_relu_layers(convNet) -> list:
  return [layer for layer in convNet if type(layer) in {nn.ReLU, nn.ReLU6}]

### Get Layers' Parameters

In [0]:
def get_conv2d_parameters(conv) -> list:
  bias = 0 if conv.bias in {None, "False"} else 1
  
  return [conv.in_channels, conv.out_channels, conv.kernel_size, conv.stride, conv.padding, conv.groups, bias]

def get_maxpool2d_parameters(maxpool) -> list:
  ceil_mode = 0 if maxpool.ceil_mode in {None, "False"} else 1

  return [maxpool.kernel_size, maxpool.padding, ceil_mode]  

def get_adaptiveavgpool2d_parameters(advpool) -> tuple:
  return advpool.output_size

def get_batchnorm2d_parameters(batchnorm) -> list:
  return [batchnorm.num_features, batchnorm.eps]

def get_dropout_parameters(dropout) -> float:
  return dropout.p

def get_linear_parameters(linear) -> list:
  return [linear.in_features, linear.out_features]

### Extract Statistics

In [0]:
def extract_conv2d_statistics(conv_layers : list) -> dict:
  conv2d_statistics = {'conv2d_total': len(conv_layers)}

  conv2d_in_channel, conv2d_out_channel, conv2d_kernel_size, conv2d_stride, conv2d_padding, conv2d_groups, conv2d_bias = [], [], [], [], [], [], []                       
  
  for conv in conv_layers:
    params = get_conv2d_parameters(conv)

    if(params[0] > 3): conv2d_in_channel.append(params[0]) #don't consider the first input channels (not informative)
    conv2d_out_channel.append(params[1])
    conv2d_kernel_size.append(params[2])
    conv2d_stride.append(params[3])
    conv2d_padding.append(params[4])
    conv2d_groups.append(params[5])
    conv2d_bias.append(params[6])

  conv2d_statistics['conv2d_in_channel_min'] = np.amin(conv2d_in_channel)
  conv2d_statistics['conv2d_in_channel_max'] = np.amax(conv2d_in_channel)
  conv2d_statistics['conv2d_in_channel_mean'] = np.mean(conv2d_in_channel)
  conv2d_statistics['conv2d_in_channel_median'] = np.median(conv2d_in_channel)
  conv2d_statistics['conv2d_in_channel_range'] = np.ptp(conv2d_in_channel)
  conv2d_statistics['conv2d_in_channel_var'] = np.var(conv2d_in_channel)
  conv2d_statistics['conv2d_in_channel_std'] = np.std(conv2d_in_channel)
  conv2d_statistics['conv2d_in_channel_mode'] = stats.mode(conv2d_in_channel)[0][0].tolist() #numpy array to list
  conv2d_statistics['conv2d_in_channel_skewness'] = stats.skew(conv2d_in_channel)
  conv2d_statistics['conv2d_in_channel_kurtosis'] = stats.kurtosis(conv2d_in_channel)

  conv2d_statistics['conv2d_out_channel_min'] = np.amin(conv2d_out_channel)
  conv2d_statistics['conv2d_out_channel_max'] = np.amax(conv2d_out_channel)
  conv2d_statistics['conv2d_out_channel_mean'] = np.mean(conv2d_out_channel)
  conv2d_statistics['conv2d_out_channel_median'] = np.median(conv2d_out_channel)
  conv2d_statistics['conv2d_out_channel_range'] = np.ptp(conv2d_out_channel)
  conv2d_statistics['conv2d_out_channel_var'] = np.var(conv2d_out_channel)
  conv2d_statistics['conv2d_out_channel_std'] = np.std(conv2d_out_channel)
  conv2d_statistics['conv2d_out_channel_mode'] = stats.mode(conv2d_out_channel)[0][0].tolist() 
  conv2d_statistics['conv2d_out_channel_skewness'] = stats.skew(conv2d_out_channel)
  conv2d_statistics['conv2d_out_channel_kurtosis'] = stats.kurtosis(conv2d_out_channel)

  conv2d_statistics['conv2d_kernel_size_min'] = list(min(conv2d_kernel_size)) #tuple to list
  conv2d_statistics['conv2d_kernel_size_max'] = list(max(conv2d_kernel_size)) 
  conv2d_statistics['conv2d_kernel_size_mode'] = stats.mode(conv2d_kernel_size)[0][0].tolist()
  
  conv2d_statistics['conv2d_stride_min'] = list(min(conv2d_stride)) 
  conv2d_statistics['conv2d_stride_max'] = list(max(conv2d_stride)) 
  conv2d_statistics['conv2d_stride_mode'] = stats.mode(conv2d_stride)[0][0].tolist()

  conv2d_statistics['conv2d_padding_min'] = list(min(conv2d_padding)) 
  conv2d_statistics['conv2d_padding_max'] = list(max(conv2d_padding)) 
  conv2d_statistics['conv2d_padding_mode'] = stats.mode(conv2d_padding)[0][0].tolist() 

  conv2d_statistics['conv2d_groups_min'] = np.amin(conv2d_groups)
  conv2d_statistics['conv2d_groups_max'] = np.amax(conv2d_groups)
  conv2d_statistics['conv2d_groups_mean'] = np.mean(conv2d_groups)
  conv2d_statistics['conv2d_groups_median'] = np.median(conv2d_groups)
  conv2d_statistics['conv2d_groups_range'] = np.ptp(conv2d_groups)
  conv2d_statistics['conv2d_groups_var'] = np.var(conv2d_groups)
  conv2d_statistics['conv2d_groups_std'] = np.std(conv2d_groups)
  conv2d_statistics['conv2d_groups_mode'] = stats.mode(conv2d_groups)[0][0].tolist()
  conv2d_statistics['conv2d_groups_skewness'] = stats.skew(conv2d_groups)
  conv2d_statistics['conv2d_groups_kurtosis'] = stats.kurtosis(conv2d_groups)

  conv2d_statistics['conv2d_bias_mode'] = stats.mode(conv2d_bias)[0][0].tolist()

  return conv2d_statistics



def extract_maxpool2d_statistics(maxpool_layers : list) -> dict:
  if(len(maxpool_layers) == 0):
    maxpool2d_statistics = {'maxpool2d_usage': 0} 
    
  else:   
    maxpool2d_statistics = {'maxpool2d_usage': 1}
    maxpool2d_statistics = {'maxpool2d_total': len(maxpool_layers)}

    maxpool2d_kernel_size, maxpool2d_padding, maxpool2d_ceil_mode = [], [], []                   
  
    for maxpool in maxpool_layers:
      params = get_maxpool2d_parameters(maxpool)
      maxpool2d_kernel_size.append(params[0])
      maxpool2d_padding.append(params[1])
      maxpool2d_ceil_mode.append(params[2])

    maxpool2d_statistics['maxpool2d_kernel_size_min'] = np.amin(maxpool2d_kernel_size)
    maxpool2d_statistics['maxpool2d_kernel_size_max'] = np.amax(maxpool2d_kernel_size)
    maxpool2d_statistics['maxpool2d_kernel_size_mean'] = np.mean(maxpool2d_kernel_size)
    maxpool2d_statistics['maxpool2d_kernel_size_median'] = np.median(maxpool2d_kernel_size)
    maxpool2d_statistics['maxpool2d_kernel_size_range'] = np.ptp(maxpool2d_kernel_size)
    maxpool2d_statistics['maxpool2d_kernel_size_var'] = np.var(maxpool2d_kernel_size)
    maxpool2d_statistics['maxpool2d_kernel_size_std'] = np.std(maxpool2d_kernel_size)
    maxpool2d_statistics['maxpool2d_kernel_size_mode'] = stats.mode(maxpool2d_kernel_size)[0][0].tolist()
    maxpool2d_statistics['maxpool2d_kernel_size_skewness'] = stats.skew(maxpool2d_kernel_size)
    maxpool2d_statistics['maxpool2d_kernel_size_kurtosis'] = stats.kurtosis(maxpool2d_kernel_size)

    maxpool2d_statistics['maxpool2d_padding_mode'] = stats.mode(maxpool2d_padding)[0][0].tolist()

    maxpool2d_statistics['maxpool2d_ceil_mode_mode'] = stats.mode(maxpool2d_ceil_mode)[0][0].tolist()

  return maxpool2d_statistics



def extract_batchnorm2d_statistics(batchnorm_layers : list) -> dict:
  if(len(batchnorm_layers) == 0):
    batchnorm2d_statistics = {'batchnorm2d_usage': 0} 

  else:
    batchnorm2d_statistics = {'batchnorm2d_usage': 1}
    batchnorm2d_statistics['batchnorm2d_total'] = len(batchnorm_layers)

    batchnorm2d_num_features, batchnorm2d_eps = [], []

    for batchnorm in batchnorm_layers:
      params = get_batchnorm2d_parameters(batchnorm)
      batchnorm2d_num_features.append(params[0])
      batchnorm2d_eps.append(params[1])

    batchnorm2d_statistics['batchnorm2d_num_features_min'] = np.amin(batchnorm2d_num_features)
    batchnorm2d_statistics['batchnorm2d_num_features_max'] = np.amax(batchnorm2d_num_features)
    batchnorm2d_statistics['batchnorm2d_num_features_mean'] = np.mean(batchnorm2d_num_features)
    batchnorm2d_statistics['batchnorm2d_num_features_median'] = np.median(batchnorm2d_num_features)
    batchnorm2d_statistics['batchnorm2d_num_features_range'] = np.ptp(batchnorm2d_num_features)
    batchnorm2d_statistics['batchnorm2d_num_features_var'] = np.var(batchnorm2d_num_features)
    batchnorm2d_statistics['batchnorm2d_num_features_std'] = np.std(batchnorm2d_num_features)
    batchnorm2d_statistics['batchnorm2d_num_features_mode'] = stats.mode(batchnorm2d_num_features)[0][0].tolist()
    batchnorm2d_statistics['batchnorm2d_num_features_skewness'] = stats.skew(batchnorm2d_num_features)
    batchnorm2d_statistics['batchnorm2d_num_features_kurtosis'] = stats.kurtosis(batchnorm2d_num_features)
    
    batchnorm2d_statistics['batchnorm2d_eps_min'] = np.amin(batchnorm2d_eps)
    batchnorm2d_statistics['batchnorm2d_eps_max'] = np.amax(batchnorm2d_eps)
    batchnorm2d_statistics['batchnorm2d_eps_mean'] = np.mean(batchnorm2d_eps)
    batchnorm2d_statistics['batchnorm2d_eps_median'] = np.median(batchnorm2d_eps)
    batchnorm2d_statistics['batchnorm2d_eps_range'] = np.ptp(batchnorm2d_eps)
    batchnorm2d_statistics['batchnorm2d_eps_var'] = np.var(batchnorm2d_eps)
    batchnorm2d_statistics['batchnorm2d_eps_std'] = np.std(batchnorm2d_eps)
    batchnorm2d_statistics['batchnorm2d_eps_mode'] = stats.mode(batchnorm2d_eps)[0][0].tolist()
    batchnorm2d_statistics['batchnorm2d_eps_skewness'] = stats.skew(batchnorm2d_eps)
    batchnorm2d_statistics['batchnorm2d_eps_kurtosis'] = stats.kurtosis(batchnorm2d_eps)

  return batchnorm2d_statistics



def extract_adaptive_avgpool2d_statistics(adaptive_avgpool2d_layers : list) -> dict:
  if(len(adaptive_avgpool2d_layers) == 0):
    adaptive_avgpool2d_statistics = {'adaptive_avgpool2d_usage': 0} 
    
  else:
    adaptive_avgpool2d_statistics = {'adaptive_avgpool2d_usage': 1}
    
    adaptive_avgpool2d_output_size = []

    for adaptive_avgpool2d in adaptive_avgpool2d_layers:
      param = get_adaptiveavgpool2d_parameters(adaptive_avgpool2d) #returns a single tuple
      adaptive_avgpool2d_output_size.append(param)
 
    adaptive_avgpool2d_statistics['adaptive_avgpool2d_output_size_mode'] = stats.mode(adaptive_avgpool2d_output_size)[0][0].tolist() 

  return adaptive_avgpool2d_statistics



def extract_dropout_statistics(dropout_layers : list) -> dict:
  if(len(dropout_layers) == 0):
    dropout_statistics = {'dropout_usage': 0} 
    
  else:
    dropout_statistics = {'dropout_usage': 1} 
    dropout_statistics['dropout_total'] = len(dropout_layers)

    dropout_p = []

    for dropout in dropout_layers:
      param = get_dropout_parameters(dropout)
      dropout_p.append(param)

    dropout_statistics['dropout_p_min'] = np.amin(dropout_p)
    dropout_statistics['dropout_p_max'] = np.amax(dropout_p)
    dropout_statistics['dropout_p_mean'] = np.mean(dropout_p)
    dropout_statistics['dropout_p_median'] = np.median(dropout_p)
    dropout_statistics['dropout_p_range'] = np.ptp(dropout_p)
    dropout_statistics['dropout_p_var'] = np.var(dropout_p)
    dropout_statistics['dropout_p_std'] = np.std(dropout_p)
    dropout_statistics['dropout_p_mode'] = stats.mode(dropout_p)[0][0].tolist()
    dropout_statistics['dropout_p_skewness'] = stats.skew(dropout_p)
    dropout_statistics['dropout_p_kurtosis'] = stats.kurtosis(dropout_p)

  return dropout_statistics



def extract_linear_statistics(linear_layers : list) -> dict:
  if(len(linear_layers) == 0):
    linear_statistics = {'linear_usage': 0} 
    
  else:
    linear_statistics = {'linear_usage': 1} 
    linear_statistics['linear_total'] = len(linear_layers)

    linear_in_features, linear_out_features = [], []

    for linear in linear_layers:
      params = get_linear_parameters(linear)
      linear_in_features.append(params[0])
      #if(index != len(linear_layers)-1): #don't consider the last output neurons (not informative) ?
      linear_out_features.append(params[1])  

    linear_statistics['linear_in_features_min'] = np.amin(linear_in_features)
    linear_statistics['linear_in_features_max'] = np.amax(linear_in_features)
    linear_statistics['linear_in_features_mean'] = np.mean(linear_in_features)
    linear_statistics['linear_in_features_median'] = np.median(linear_in_features)
    linear_statistics['linear_in_features_range'] = np.ptp(linear_in_features)
    linear_statistics['linear_in_features_var'] = np.var(linear_in_features)
    linear_statistics['linear_in_features_std'] = np.std(linear_in_features)
    linear_statistics['linear_in_features_mode'] = stats.mode(linear_in_features)[0][0].tolist()
    linear_statistics['linear_in_features_skewness'] = stats.skew(linear_in_features)
    linear_statistics['linear_in_features_kurtosis'] = stats.kurtosis(linear_in_features)

    linear_statistics['linear_out_features_min'] = np.amin(linear_out_features)
    linear_statistics['linear_out_features_max'] = np.amax(linear_out_features)
    linear_statistics['linear_out_features_mean'] = np.mean(linear_out_features)
    linear_statistics['linear_out_features_median'] = np.median(linear_out_features)
    linear_statistics['linear_out_features_range'] = np.ptp(linear_out_features)
    linear_statistics['linear_out_features_var'] = np.var(linear_out_features)
    linear_statistics['linear_out_features_std'] = np.std(linear_out_features)
    linear_statistics['linear_out_features_mode'] = stats.mode(linear_out_features)[0][0].tolist()
    linear_statistics['linear_out_features_skewness'] = stats.skew(linear_out_features)
    linear_statistics['linear_out_features_kurtosis'] = stats.kurtosis(linear_out_features)    

  return linear_statistics



def extract_relu_statistics(relu_layers : list) -> dict:
  return {'relu_total': len(relu_layers)}

# **Main**

In [0]:
#12 different ConvNets, 31 in total
convNet_name_list = ["alexnet", "googlenet", "mobilenet_v2", "vgg11", "vgg11_bn", "vgg13", "vgg13_bn", "vgg16", "vgg16_bn", "vgg19", "vgg19_bn", "resnet18", "resnet34", "resnet50", "resnet101", 
            "resnet152", "squeezenet1_0", "squeezenet1_1", "densenet121", "densenet161", "densenet169", "densenet201", "shufflenet_v2_x0_5", "shufflenet_v2_x1_0", "resnext50_32x4d", "resnext101_32x8d",
            "wide_resnet50_2", "wide_resnet101_2", "mnasnet0_5", "mnasnet1_0", "inception_v3"]

#convNet_name_list = ["alexnet", "vgg13_bn"]

In [0]:
for convNet_name in convNet_name_list:
  print("\n\n############################################\n", convNet_name, "\n############################################\n")

  convNet = call_constructor(convNet_name)
  layers_all = [module for module in convNet.modules() if type(module) in {nn.Conv2d, nn.MaxPool2d, nn.AdaptiveAvgPool2d, nn.BatchNorm2d, nn.Dropout, nn.Linear, nn.ReLU, nn.ReLU6}]
  print(layers_all, "\n")

  conv_all = get_conv2d_layers(layers_all)
  maxpool_all = get_maxpool2d_layers(layers_all)
  adaptive_avgpool_all = get_adaptive_avgpool2d_layers(layers_all)
  batchnorm_all = get_batchnorm2d_layers(layers_all)
  dropout_all = get_dropout_layers(layers_all)
  linear_all = get_linear_layers(layers_all)
  relu_all = get_relu_layers(layers_all)
  
  conv_stats = extract_conv2d_statistics(conv_all)
  maxpool_stats = extract_maxpool2d_statistics(maxpool_all)
  adaptive_avgpool_stats = extract_adaptive_avgpool2d_statistics(adaptive_avgpool_all)
  batchnorm_stats = extract_batchnorm2d_statistics(batchnorm_all)
  dropout_stats = extract_dropout_statistics(dropout_all)
  linear_stats = extract_linear_statistics(linear_all)
  relu_all = extract_relu_statistics(relu_all)

  model_metafeatures = {**conv_stats, **maxpool_stats, **adaptive_avgpool_stats, **batchnorm_stats, **dropout_stats, **linear_stats, **relu_all}
  print(model_metafeatures, "\n")



############################################
 alexnet 
############################################

[Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2)), ReLU(inplace=True), MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False), Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2)), ReLU(inplace=True), MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False), Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace=True), Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace=True), Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace=True), MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False), AdaptiveAvgPool2d(output_size=(6, 6)), Dropout(p=0.5, inplace=False), Linear(in_features=9216, out_features=4096, bias=True), ReLU(inplace=True), Dropout(p=0.5, inplace=False), Linear(in_features=4096, out_features=4096, bias

# **DataFrame creation**

In [0]:
df = pd.DataFrame(data=model_metafeatures)

print(len(model_metafeatures))
print(model_metafeatures)
df

89
{'conv2d_total': 96, 'conv2d_in_channel_min': 32, 'conv2d_in_channel_max': 2048, 'conv2d_in_channel_mean': 432.8421052631579, 'conv2d_in_channel_median': 192.0, 'conv2d_in_channel_range': 2016, 'conv2d_in_channel_var': 211479.54349030473, 'conv2d_in_channel_std': 459.8690503722823, 'conv2d_in_channel_mode': 768, 'conv2d_in_channel_skewness': 2.0104879451574087, 'conv2d_in_channel_kurtosis': 4.05611327731681, 'conv2d_out_channel_min': 32, 'conv2d_out_channel_max': 768, 'conv2d_out_channel_mean': 188.66666666666666, 'conv2d_out_channel_median': 192.0, 'conv2d_out_channel_range': 736, 'conv2d_out_channel_var': 15306.222222222224, 'conv2d_out_channel_std': 123.7183180544507, 'conv2d_out_channel_mode': 192, 'conv2d_out_channel_skewness': 1.5316402205718402, 'conv2d_out_channel_kurtosis': 3.6010655031615793, 'conv2d_kernel_size_min': [1, 1], 'conv2d_kernel_size_max': [7, 1], 'conv2d_kernel_size_mode': [1, 1], 'conv2d_stride_min': [1, 1], 'conv2d_stride_max': [2, 2], 'conv2d_stride_mode': 

,conv2d_total,conv2d_in_channel_min,conv2d_in_channel_max,conv2d_in_channel_mean,conv2d_in_channel_median,conv2d_in_channel_range,conv2d_in_channel_var,conv2d_in_channel_std,conv2d_in_channel_mode,conv2d_in_channel_skewness,conv2d_in_channel_kurtosis,conv2d_out_channel_min,conv2d_out_channel_max,conv2d_out_channel_mean,conv2d_out_channel_median,conv2d_out_channel_range,conv2d_out_channel_var,conv2d_out_channel_std,conv2d_out_channel_mode,conv2d_out_channel_skewness,conv2d_out_channel_kurtosis,conv2d_kernel_size_min,conv2d_kernel_size_max,conv2d_kernel_size_mode,conv2d_stride_min,conv2d_stride_max,conv2d_stride_mode,conv2d_padding_min,conv2d_padding_max,conv2d_padding_mode,conv2d_groups_min,conv2d_groups_max,conv2d_groups_mean,conv2d_groups_median,conv2d_groups_range,conv2d_groups_var,conv2d_groups_std,conv2d_groups_mode,conv2d_groups_skewness,conv2d_groups_kurtosis,...,batchnorm2d_num_features_range,batchnorm2d_num_features_var,batchnorm2d_num_features_std,batchnorm2d_num_features_mode,batchnorm2d_num_features_skewness,batchnorm2d_num_features_kurtosis,batchnorm2d_eps_min,batchnorm2d_eps_max,batchnorm2d_eps_mean,batchnorm2d_eps_median,batchnorm2d_eps_range,batchnorm2d_eps_var,batchnorm2d_eps_std,batchnorm2d_eps_mode,batchnorm2d_eps_skewness,batchnorm2d_eps_kurtosis,dropout_usage,linear_usage,linear_total,linear_in_features_min,linear_in_features_max,linear_in_features_mean,linear_in_features_median,linear_in_features_range,linear_in_features_var,linear_in_features_std,linear_in_features_mode,linear_in_features_skewness,linear_in_features_kurtosis,linear_out_features_min,linear_out_features_max,linear_out_features_mean,linear_out_features_median,linear_out_features_range,linear_out_features_var,linear_out_features_std,linear_out_features_mode,linear_out_features_skewness,linear_out_features_kurtosis,relu_total
0,96,32,2048,432.842105,192.0,2016,211479.54349,459.86905,768,2.010488,4.056113,32,768,188.666667,192.0,736,15306.222222,123.718318,192,1.53164,3.601066,1,7,1,1,2,1,0,3,0,1,1,1.0,1.0,0,0.0,0.0,1,0.0,-3.0,...,736,15306.222222,123.718318,192,1.53164,3.601066,0.001,0.001,0.001,0.001,0.0,4.701977e-38,2.168404e-19,0.001,-1.0,-2.0,0,1,2,768,2048,1408.0,1408.0,1280,409600.0,640.0,768,0.0,-2.0,1000,1000,1000.0,1000.0,0,0.0,0.0,1000,0.0,-3.0,0
1,96,32,2048,432.842105,192.0,2016,211479.54349,459.86905,768,2.010488,4.056113,32,768,188.666667,192.0,736,15306.222222,123.718318,192,1.53164,3.601066,1,1,1,1,2,1,0,0,0,1,1,1.0,1.0,0,0.0,0.0,1,0.0,-3.0,...,736,15306.222222,123.718318,192,1.53164,3.601066,0.001,0.001,0.001,0.001,0.0,4.701977e-38,2.168404e-19,0.001,-1.0,-2.0,0,1,2,768,2048,1408.0,1408.0,1280,409600.0,640.0,768,0.0,-2.0,1000,1000,1000.0,1000.0,0,0.0,0.0,1000,0.0,-3.0,0


In [0]:
df = df.drop_duplicates()
df

,conv2d_total,conv2d_in_channel_min,conv2d_in_channel_max,conv2d_in_channel_mean,conv2d_in_channel_median,conv2d_in_channel_range,conv2d_in_channel_var,conv2d_in_channel_std,conv2d_in_channel_mode,conv2d_in_channel_skewness,conv2d_in_channel_kurtosis,conv2d_out_channel_min,conv2d_out_channel_max,conv2d_out_channel_mean,conv2d_out_channel_median,conv2d_out_channel_range,conv2d_out_channel_var,conv2d_out_channel_std,conv2d_out_channel_mode,conv2d_out_channel_skewness,conv2d_out_channel_kurtosis,conv2d_kernel_size_min,conv2d_kernel_size_max,conv2d_kernel_size_mode,conv2d_stride_min,conv2d_stride_max,conv2d_stride_mode,conv2d_padding_min,conv2d_padding_max,conv2d_padding_mode,conv2d_groups_min,conv2d_groups_max,conv2d_groups_mean,conv2d_groups_median,conv2d_groups_range,conv2d_groups_var,conv2d_groups_std,conv2d_groups_mode,conv2d_groups_skewness,conv2d_groups_kurtosis,...,batchnorm2d_num_features_range,batchnorm2d_num_features_var,batchnorm2d_num_features_std,batchnorm2d_num_features_mode,batchnorm2d_num_features_skewness,batchnorm2d_num_features_kurtosis,batchnorm2d_eps_min,batchnorm2d_eps_max,batchnorm2d_eps_mean,batchnorm2d_eps_median,batchnorm2d_eps_range,batchnorm2d_eps_var,batchnorm2d_eps_std,batchnorm2d_eps_mode,batchnorm2d_eps_skewness,batchnorm2d_eps_kurtosis,dropout_usage,linear_usage,linear_total,linear_in_features_min,linear_in_features_max,linear_in_features_mean,linear_in_features_median,linear_in_features_range,linear_in_features_var,linear_in_features_std,linear_in_features_mode,linear_in_features_skewness,linear_in_features_kurtosis,linear_out_features_min,linear_out_features_max,linear_out_features_mean,linear_out_features_median,linear_out_features_range,linear_out_features_var,linear_out_features_std,linear_out_features_mode,linear_out_features_skewness,linear_out_features_kurtosis,relu_total
0,96,32,2048,432.842105,192.0,2016,211479.54349,459.86905,768,2.010488,4.056113,32,768,188.666667,192.0,736,15306.222222,123.718318,192,1.53164,3.601066,1,7,1,1,2,1,0,3,0,1,1,1.0,1.0,0,0.0,0.0,1,0.0,-3.0,...,736,15306.222222,123.718318,192,1.53164,3.601066,0.001,0.001,0.001,0.001,0.0,4.701977e-38,2.168404e-19,0.001,-1.0,-2.0,0,1,2,768,2048,1408.0,1408.0,1280,409600.0,640.0,768,0.0,-2.0,1000,1000,1000.0,1000.0,0,0.0,0.0,1000,0.0,-3.0,0
1,96,32,2048,432.842105,192.0,2016,211479.54349,459.86905,768,2.010488,4.056113,32,768,188.666667,192.0,736,15306.222222,123.718318,192,1.53164,3.601066,1,1,1,1,2,1,0,0,0,1,1,1.0,1.0,0,0.0,0.0,1,0.0,-3.0,...,736,15306.222222,123.718318,192,1.53164,3.601066,0.001,0.001,0.001,0.001,0.0,4.701977e-38,2.168404e-19,0.001,-1.0,-2.0,0,1,2,768,2048,1408.0,1408.0,1280,409600.0,640.0,768,0.0,-2.0,1000,1000,1000.0,1000.0,0,0.0,0.0,1000,0.0,-3.0,0


In [0]:
df.to_csv("model_metafeatures.csv", index=False, float_format='%.6f')

# **Debug**

In [0]:
'''
print("convNet: ", convNet)
print("all layers: ", layers_all)
print("conv_all: ", conv_all)
print(conv_all_parameters)  
print("conv stats: ", extract_conv2d_statistics(conv_all))
print("maxpool_all: ", maxpool_all)
print(maxpool_all_parameters)
print("adaptive_avgpool_all: ", adaptive_avgpool_all)
print(adaptive_avgpool_all_parameters)
print("batchnorm_all: ", batchnorm_all)
print(batchnorm_all_parameters)
print("dropout_all: ", dropout_all)
print(dropout_all_parameters)
print("linear_all: ", linear_all)
print(linear_all_parameters)
print("relu_all: ", relu_all)
print(relu_all_parameters)  
'''